In [1]:
import numpy as np
import hemul
hemul.USE_FPGA=False
from hemul import heaan
from muxcnn.resnet_HEAAN import ResNetHEAAN
from muxcnn.utils import get_channel_last

Using CPU version HEAAN
Using CPU version HEAAN
Using CPU version HEAAN


## Torch model

In [2]:
#import torch
from muxcnn.models.ResNet20 import ResNet, BasicBlock
from muxcnn.utils import load_params, load_img

# 원본 모델
model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

### 예제 이미지 load

In [3]:
img_tensor = load_img("./cute.jpg", hi=32, wi=32)

### Tensor 모양 변경
imgl = get_channel_last(img_tensor[0].detach().numpy())

# class에 default 값 저장되어있음 
#hi, wi, ch = imgl.shape
#ki = 1 # initial ki - multiplex parameter

# FHE

In [4]:
logp = 40
logq = 800
logn = 15

# Rotation 미리 준비 
rot_l = [2**i for i in range(15)]

# prepare for all rotations
rot_l = rot_l + [2**15-1, 
         2**15-33, 2**15-32, 2**15-31,
         2**15-17, 2**15-16, 2**15-15, 
         2**15-9,2**15-8, 2**15-7] + [3,5,7,9,15,17, 31, 33]

In [5]:
# Context & Evaluator
hec = heaan.HEAANContext(logn, logp, logq, boot=False, load_sk=True, rot_l=rot_l)

Initializing the scheme as the data owner
Loading a secret key from:  ./SecretKey.txt
loading secret key done.
HEAAN CKKS setup is ready 


In [6]:
fhemodel = ResNetHEAAN(model, hec, alpha=17)

# 이미지 암호화 
ctxt = fhemodel.pack_img_ctxt(img_tensor)

functions set
degrees = [15, 15, 15, 29, 29], margin = 0.03, eps = 0.01


/home/hoseung/anaconda3/envs/cnn/lib/python3.10/site-packages/numpy/polynomial/chebyshev.py:1671: RankWarning: The fit may be poorly conditioned
  return pu._fit(chebvander, x, y, deg, rcond, full, w)


## Inference

In [7]:
debug = True
ki, hi, wi = 1, 32, 32

ctxt0, outs0 = fhemodel.forward_early(ctxt, ki, hi, wi)
#self.hec.rescale(ctxt)
if debug: print(" - - - - After early", ctxt)
# Basic blocks
ctxt1, outs1 = fhemodel.forward_bb(model.layer1[0], ctxt0, outs0)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
ct_a 40 800 [-7.49019623e-01 -9.13725497e-01 -5.52941206e-01 -8.43137265e-01
  4.54937187e-09 -4.11764680e-01  9.80392743e-02 -8.50980400e-01
 -5.19155508e-09 -2.47058808e-01  1.29411810e-01 -6.78431395e-01
 -5.15202333e-09  8.23530039e-02  2.94117685e-01  2.31372592e-01
  3.93488731e-09  1.92156908e-01  1.52941251e-01  1.29411814e-01
  3.36732685e-09  2.78431416e-01 -5.52941207e-01 -5.45098065e-01
 -5.27612692e-09  1.05882407e-01  4.98039246e-01  4.50980420e-01
 -2.31019411e-10  7.41176491e-01  7.41176490e-01 -4.27450956e-01
  2.79072153e-09]
ct_d 40 720
temp  ----   40 800 [-1.72435489e-08 -8.50980430e-01 -4.58823497e-01 -7.49019536e-01
  4.57991654e-08 -1.68627432e-01  9.01961591e-02 -8.11764735e-01
 -7.91610685e-10 -1.45098027e-01  1.29411826e-01  3.17647068e-01
  1.52174830e-08  1.13725525e-01 -5.05882438e-01  9.13725524e-01
 -4.84026701e-08  3.098039

temp  ----   40 240 [ 1.45467315e-03  2.42429218e-01  7.06004045e-01  8.15911191e-05
  1.36810276e+00 -2.51463721e-04  5.72253211e-04  2.16009597e-01
  3.94266134e-04 -6.67666438e-05 -1.23455249e-03  7.78537747e-04
 -7.88810936e-04  1.78293017e-01  1.79204409e-02  9.42450315e-01
  8.38848151e-04  1.68964930e-04  4.42905580e-01  5.54659080e+00
  8.46320511e-01  3.06645017e-04  1.32058959e+00  2.71624308e+00
 -3.20728401e-04  2.62556104e-01  1.44623582e-04  1.88513533e-03
  3.88046744e-03  2.15033040e-01  6.65864929e-01  7.78147693e-02
 -3.31442555e-04]
temp  ----   40 240 [ 4.13683655e-03  2.40288977e-01  7.49565969e-01  1.05945517e-01
  7.55517463e-05  5.20168411e-04  3.26954881e-04  2.10609973e-01
  3.16257241e-01  4.55809019e-05  8.65012659e-05 -5.68703263e-05
  5.43190786e-01  8.65792356e-02  1.07458410e-01  1.29736007e+00
 -7.04957800e-05  1.31796963e-02  4.40241725e-01  3.23512334e+00
  9.18882992e-04  1.42766804e-01  1.93570228e+00  2.88014471e+00
  2.49187567e-01  1.15534880e-02

temp  ----   40 120 [ 9.44383016e-05 -1.48220015e-03 -3.33158906e-03  4.50977753e-04
  1.15432042e+00  1.11530856e-03 -1.66228196e-04 -1.41135586e-03
 -1.84245069e-04  3.80205269e-04  3.28474389e-01  1.55858853e+00
 -6.99608395e-03 -9.23604617e-05  1.12295505e+00 -9.05977792e-04
 -2.81610856e-04 -3.79514932e-04  4.00743762e-04  6.35735490e-05
  1.11199926e+00  1.24650004e-03  1.92248875e-04  2.46987157e-03
  1.03079614e-03  6.32127781e-01  3.46220131e-01  1.31805923e+00
  3.96410495e-04  5.84763632e-01  1.92250985e-03  1.87498241e+00
 -8.54115708e-04]
temp  ----   40 120 [ 8.63146038e-01  4.03980225e-04 -5.22151993e-04  1.45868457e-04
  1.68722619e+00  5.76030696e-04 -2.36311972e-04  5.92661984e-04
  1.50267151e+00  4.88981283e-04  3.69591488e-01 -3.11883914e-05
  6.54104238e-04  1.49010843e-01 -4.67172432e-04  8.96821120e-02
  7.77457390e-01  1.31975344e-04 -5.55934505e-04  3.19224447e-03
  1.42653102e+00  3.72808016e-04 -3.39195054e-04 -2.67907921e-04
  1.46999533e+00  4.44034479e-01

In [21]:
if debug: print("After first block\n\n")
ctxt2, outs2 = fhemodel.forward_bb(model.layer2[0], ctxt1, outs1)

After first block


ZZZZZZZZZZZZZZZZZZZZZZ  11
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
ct_a 40 240 [ 1.05783842e+00  2.36732359e-01  1.64305331e-01  2.23620914e-04
  3.12632881e-07 -1.36105883e-03  1.00547187e+00 -5.24574696e-05
  1.78355608e+00  1.86545041e-06 -4.76985068e-05  4.12514025e-01
  1.17867781e-03  2.49556099e-01  1.06664703e+00  2.26746715e-01
  6.92812239e-01  4.75884144e-02  9.16121694e-01  4.27398283e-03
 -8.96123306e-04 -3.99225178e-05  1.01063013e-03 -5.92489511e-03
  1.04177261e+00  3.03823378e-04 -2.15090843e-03 -1.01869799e-03
 -8.21271250e-04  1.86322819e-03  1.34704915e+00 -3.84613529e-04
  6.82550343e-01]
ct_d 40 160
temp  ----   40 240 [ 6.85402162e-01  1.91107171e-01  6.48916412e-01 -1.25400447e-03
  7.21746568e-01  3.97822356e-04 -2.14880593e-03  5.31563998e-04
  1.39235654e+00  3.91368301e-04  3.94189350e-01  3.38099467e-01
  2.37486019e-03  9.37250299e-05  1.4244004

temp  ----   40 120 [ 1.06349593e+00  7.60315716e-01  3.41055619e-01  9.77866885e-01
  1.17714146e+00 -4.73604223e-05  1.15613368e+00  1.34846216e+00
  1.11144449e+00  9.95966066e-01  2.76782132e-01 -3.26934826e-05
  1.08250879e+00  6.07089026e-01  6.80080071e-01 -1.92819701e-04
  9.49273188e-01  5.69330348e-01  4.96059377e-01  4.33613581e-05
  1.10464935e+00  3.09527121e-01  1.94567130e+00  3.59749109e-05
  1.07155320e+00  4.97995604e-01  6.42747877e-01 -1.99493153e-04
  1.12300700e+00  7.28588836e-01  1.44580854e+00 -1.33480104e-04
  1.01234831e+00]
temp  ----   40 120 [ 1.41742938e+00  3.40423137e-01  4.25361344e-02  3.53519619e-01
  6.75757810e-01  4.58367577e-01  1.57662901e+00  6.57132825e-01
  2.09831528e+00  2.74538295e-01  6.92431411e-01  6.04158493e-05
  4.03627773e-01  5.41034184e-01  1.09428167e+00 -7.30805930e-05
  1.90392294e+00  3.34366152e-01  7.52235259e-01  8.55140780e-01
  5.61475443e-01  8.28255302e-01  1.69338948e+00  2.93337290e-06
  1.96859615e+00  5.89211744e-01

[ 1.11902237e-01  1.37929540e+00  1.77675924e-04 ...  8.42776980e-04
 -4.14227768e-04 -3.47188671e-06]
min max -0.005042038255225896 7.225756319828167


In [ ]:
fhemodel = ResNetHEAAN(model, hec, alpha=17)

In [15]:
from muxcnn.comparator_heaan import ApprRelu_HEAAN

In [22]:
fhemodel._set_activation(alpha = 17, xmin=-50, xmax=50, min_depth=True, eps=0.01, margin=0.04)

functions set
degrees = [15, 15, 15, 29, 29], margin = 0.04, eps = 0.01


In [ ]:
if debug: print("After second block\n\n")
ctxt3, outs3 = fhemodel.forward_bb(model.layer3[0], ctxt2, outs2)

After second block


ZZZZZZZZZZZZZZZZZZZZZZ  11
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =( 8, 8,64, 4, 4,  8)
[MultParConv] q = 16
ct_a 40 240 [1.11902237e-01 6.70280009e-01 4.23457232e-01 6.25870952e-02
 6.82461323e-01 2.31679391e+00 5.21744012e-01 1.30025366e+00
 9.47832632e-02 5.72305141e-05 1.04613440e+00 1.71299038e+00
 1.05245484e+00 3.54249877e+00 1.16480494e+00 7.91184892e-01
 7.50557988e-06 1.65393189e-04 7.00346481e-02 1.62493149e-01
 1.09523469e+00 3.02092990e+00 5.07611599e-01 2.71784437e-05
 3.00430124e-01 9.27566459e-02 4.11322753e-01 5.48371626e-01
 3.85416113e-01 2.92592091e+00 4.71103836e-01 1.32034617e+00
 3.95719242e-01]
ct_d 40 160
temp  ----   40 240 [ 4.43369573e-04  6.37151416e-01  1.16658824e+00  1.15783173e+00
  2.67490205e+00  2.26485995e+00  1.00089930e+00  1.17745997e+00
  2.52173066e-04  1.24892079e-01  1.11743473e+00  2.18228710e+00
  2.59236657e+00  3.32131616e+00  5.90875382e-01  8.65486880e-01
 -1.860722

In [13]:
if debug: print("After second block\n\n")
ctxt3, outs3 = fhemodel.forward_bb(model.layer3[0], ctxt2, outs2)

After second block


ZZZZZZZZZZZZZZZZZZZZZZ  11
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =( 8, 8,64, 4, 4,  8)
[MultParConv] q = 16
ct_a 40 240 [ 1.12374479e-01  6.68310397e-01  4.23416837e-01  6.46579559e-02
  6.83886780e-01  2.31477786e+00  5.24525851e-01  1.29910921e+00
  9.16952964e-02  1.53763157e-04  1.04788323e+00  1.71079380e+00
  1.05126349e+00  3.54313524e+00  1.16316272e+00  7.92508561e-01
 -3.42312765e-04 -1.32612712e-04  6.91886696e-02  1.65384689e-01
  1.09478367e+00  3.02172328e+00  5.02477807e-01 -3.64319458e-04
  3.01096257e-01  9.36125985e-02  4.14332788e-01  5.47499426e-01
  3.85228524e-01  2.92866335e+00  4.73756760e-01  1.32136869e+00
  3.96631083e-01]
ct_d 40 160
temp  ----   40 240 [ 1.05403316e-03  6.37436807e-01  1.16560535e+00  1.16032905e+00
  2.67838877e+00  2.26772608e+00  1.00354293e+00  1.18441373e+00
  2.28973398e-03  1.25954068e-01  1.11682849e+00  2.17992703e+00
  2.59768589e+00  3.30788747e+00  5.881864

temp  ----   40 120 [ 2.95647516e-04  2.50606227e+00  1.63842568e+00  5.57951518e+00
 -1.08185619e-04  6.06183405e-01  4.21709446e+00  2.11456987e+00
  1.68205031e-03  4.82092170e+00 -4.79372212e-04  1.25077572e+00
  3.85657485e+00  1.70246886e+00  2.11688523e+00  7.65877535e-01
 -3.52509926e-04  1.78832009e+00  1.82724968e+00  5.55467003e+00
  3.03844554e-04  1.90553777e-01  4.32906629e+00  1.77669155e+00
  6.21687729e-04  4.64763098e+00  2.81924922e-03  2.13070249e+00
  4.44023393e+00  1.37224899e+00  2.86938368e+00  9.20346645e-01
  9.94903253e-01]
temp  ----   40 120 [ 2.24825459e+00  1.66337670e+00  1.09772529e+00  5.54465132e+00
  3.22458007e-01  4.11994635e-01  4.34927004e+00  1.69280546e+00
  3.38611077e+00  5.23654534e+00 -2.58237281e-04  1.95398731e+00
  3.78262959e-01  8.96523747e-01  3.42070684e+00  1.76107652e+00
  1.45889171e+00  1.92209017e+00  2.20936418e+00  6.74215038e+00
  2.13960999e-01  6.70851564e-01  3.76690378e+00  3.82840080e-02
  3.86644494e+00  5.71275926e+00

min max -3.5661720189793893e+74 3.902301784273423e+74
<class.Ciphertext logp: 40 logq: 800 n: 32768> <class.Ciphertext logp: 40 logq: 280 n: 32768>
After mult 80 280
[-1.49122420e+62 -3.54298596e+61 -2.44509805e+61 ... -6.37640693e+61
 -7.15945335e+61 -1.47300703e+61]
min max -3.5121669312256395e+62 3.4598025960133953e+62


In [ ]:
if debug: print("After third block\n\n")
ctxt = fhemodel.AVGPool(ctxt, outs3, self.nslots) # Gloval pooling
fhemodel.forward_linear(ctxt, model.linear)

In [7]:
result = fhemodel(ctxt)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
APPR 15 40 720
[ 1.23083763  0.48696443  0.2753431  ... -0.65056951 -0.61289198
 -0.4793408 ]
min max -5.745771076952116 5.543758043807229
APPR 15 40 520
[ 0.25284941  0.10078295  0.05703897 ... -0.13449721 -0.12674295
 -0.09920941]
min max -0.9834750495944412 0.9614492948673184
APPR 15 40 320
[ 1.1628899   0.77293312  0.46823725 ... -0.9542947  -0.91728077
 -0.76322391]
min max -1.170108620466743 1.1701088630712326
APPR 29 40 120
[ 1.00768405  1.00085105  0.9929096  ... -1.00366711 -0.99794165
 -1.0022668 ]
min max -1.0165753873786685 1.0165753760687644
bootstrap done
AFTER bootstrap 40 800
APPR 29 40 560
[ 1.00000474  0.99999907  1.00000492 ... -0.99999563 -1.00000527
 -0.99999273]
min max -1.000042884630429 1.0000399139215923
After ApprSign 40 320
[ 0.99997933  1.00002391  0.99998497 ... -1.00006756 -0.99993653
 -0.99992486]
min max -1.0005535208189473 

KeyboardInterrupt: 

BB forward
결과 np 버전이랑 비교.  

문제 없으면 마지막 layer는 np 버전에서 가져온 뒤에 encrypt해서 실행 

In [ ]:
# 결과 확인 
dec = hec.decrypt(result)
print("Class =", np.argmax(dec[:10]))